# Object detection with the help of depth camera
Pros:
No need to have contrast background
Working code!!

In [3]:
import cv2
import pyrealsense2
from pylogix import PLC
import keyboard
import time
from realsense_aligned_depth import *




def Sort(s):
  
    # reverse = None (Sorts in Ascending order)
    # key is set to sort using second element of 
    # sublist lambda has been used
    s.sort(key = lambda x: x[0])
    return s


#initialise the depth camera
dc = DepthCamera()
first_frame= True
list=[0]
np.array(list)


while first_frame is True:
    ret,depth_frame,color_frame= dc.get_frame()

    #point=(400,300)
    #cv2.circle(color_frame,point,4,(0,0,255))
    #distance = depth_frame[point[1], point[0]] #first height then width
    #print(distance)
    
    sval=60000
    for i in range(125,275):
        for j in range(150,401):
            if sval>depth_frame[i][j] & depth_frame[i][j] !=0:
                sval=depth_frame[i][j]
    if len(list)>50:
        list.pop(0)
        list.append(sval)
    else:
        list.append(sval)

    cv2.imshow("color_frame", color_frame)
    cv2.imshow("depth_frame", depth_frame)
    
    if cv2.waitKey(1) & 0xFF == ord('c'):
        avg_sval= round(sum(list)/len(list))
        print(avg_sval)
        ff_d=depth_frame
        nf_color=color_frame
        #cv2.imwrite("c1.png",bg_gray)
        
#         sval=3000
#         for i in range(50,400):
#             for j in range(50,550):
#                 if sval>ff_d[i][j] & ff_d[i][j] !=0:
#                     sval=ff_d[i][j]

        # Getting the depth sensor's depth scale (see rs-align example for explanation)
        #depth_sensor = profile.get_device().first_depth_sensor()
        depth_scale = 0.0010000000474974513 #depth_sensor.get_depth_scale()
        # We will be removing the background of objects more than
        #  clipping_distance_in_meters meters away
        clipping_distance_in_meters = (avg_sval+60)/1000 #1 meter
        clipping_distance = clipping_distance_in_meters / depth_scale
         # Remove background - Set pixels further than clipping_distance to grey
        grey_color = 0
        depth_image_3d = np.dstack((ff_d,ff_d,ff_d)) #depth image is 1 channel, color is 3 channels
        bg_removed = np.where((depth_image_3d > clipping_distance) | (depth_image_3d <= 0), grey_color, nf_color)

        bilateral = cv2.bilateralFilter(bg_removed, 15, 60, 60) #Bilateral filter
        gray = cv2.cvtColor(bilateral, cv2.COLOR_BGR2GRAY) # Gray filter
        ret, th1 = cv2.threshold(gray,90,255,cv2.THRESH_BINARY) #thresolding

        kernel=np.ones((3,3),np.uint8)#cv2.getStructuringElement(cv2.MORPH_CROSS,(9,9))
        Open= cv2.morphologyEx(th1,cv2.MORPH_OPEN, kernel,iterations=10) #Morphing(open=erode+dilate)

        contours, hierarchy = cv2.findContours(Open, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(nf_color, contours, -1, (250,0,0), 1)
        arr=[]
        for i in range (len(contours)):
            cnt2 = contours[i]
            area=cv2.contourArea(cnt2)
            AREA = 2500 #cv2.getTrackbarPos("AREA", "Parameter")#for trackbar
            if area>=AREA:
                #epsilon = 0.1*cv2.arcLength(cnt2,True)
                #approx = cv2.approxPolyDP(cnt2,epsilon,True)
                rect = cv2.minAreaRect(cnt2) #returns [rectangle center(x, y), (width, height), rotation angle)].
                box = cv2.boxPoints(rect)
                box = np.int0(box)
                color_frame = cv2.drawContours(nf_color,[box],0,(0,255,255),2)
                #M = cv2.moments(cnt2)
                #cx2 = int(M['m10']/M['m00'])
                #cy2 = int(M['m01']/M['m00'])
                cx2=int(rect[0][0])
                cy2=int(rect[0][1])
                point = (cx2 , cy2)

                width= int(rect[1][0])
                height= int(rect[1][1])

                 # Show distance for a specific point
                cv2.circle(nf_color, point, 4, (0, 0, 255))
                distance =ff_d[point[1],point[0]] #avg_sval ##nf_depth[240, 320] # [point[1],point[0]]

                dist=distance/10          #input in cm
                cv2.putText(nf_color, "{}cm".format(dist), (point[0], point[1] - 20), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2)

                cvtfac=0.00304535*distance**(0.921814) -0.17026 #0.00167228*distance -0.00381802
                WT=(round(width*cvtfac,2))
                H8=(round(height*cvtfac,2))
                #print(width,height)

                #Pixel_to_MM

                Rx=(round(cx2*cvtfac,2))
                Ry=(round(cy2*cvtfac,2))


                #color_frame=cv2.circle(color_frame, (cx2, cy2), 7, (0, 100, 200), -1)
                angle=rect[-1]
                #print(angle)
                if angle<-45:
                    angle=-(90+angle)
                else:
                    angle=-angle
                #print(angle,"deg")
                x_dis=abs(cx2-box[0,0]) #dist btw x value of center of contour and x-value of first vertex of contour
                y_dis=abs(cy2-box[0,1])#dist btw y value of center of contour and y-value of first vertex of contour

                if (x_dis<y_dis and (angle<10 and angle>-10)):
                    nf_color=cv2.putText(nf_color,"v"+str( WT)+","+str( H8) , (cx2-10, cy2-10), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 0, 155), 2)
                    orientation=900
                    #print("its verticle")
                elif (x_dis>y_dis and (angle<10 and angle>-10)):
                    orientation=0
                    #print("its Horizontal")
                    nf_color=cv2.putText(nf_color,"H"+str(WT)+","+str(H8), (cx2-10, cy2-10), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 155, 0), 2)
                else:
                    nf_color=cv2.putText(nf_color,"I"+str( WT)+","+str(H8) , (cx2-10, cy2-10), cv2.FONT_HERSHEY_SIMPLEX,0.5, (155, 0, 0), 2)
                    orientation=450
                    #print("Its inclined")
                #To ROBOT
                robx=(round((cx2-320)*cvtfac,0))
                roby=(round((240-cy2)*cvtfac,0))


                arr.append([robx,roby, -1805+distance,0,0 ,orientation])#1795
                
        print(arr)
        cv2.imshow("Showing contour", nf_color)
        k=cv2.waitKey(0)
        arr=Sort(arr)
        print(arr)
        for i in range(len(arr)):
            comm =PLC()
            comm.IPAddress = '192.168.255.2'
            n=comm.Read('pulse_to_pc')
            print(n.Value)
            real_x=arr[i][0]
            real_y=arr[i][1]
            real_z=arr[i][2]
            real_rx=arr[i][3]
            real_ry=arr[i][4]
            real_rz=arr[i][5]
            print("Grab item from x:" +str(real_y)+", y:"+str(real_x)+", z:"+str(real_z)+", Rz:"+str(real_rz))
            comm.IPAddress = '192.168.255.2'
            comm.Write('x_axis_frm_pc', real_y)
            comm.Write('y_axis_frm_pc', real_x)
            comm.Write('z_axis_frm_pc', real_z)
            comm.Write('r_x_frm_pc', real_rx)
            comm.Write('r_y_frm_pc', real_ry)
            comm.Write('r_z_frm_pc', real_rz)
            comm.Close()
            #time.sleep(20)
            while n.Value is False:
                #print(n.Value)
                n=comm.Read('pulse_to_pc')# should get pulse of '1'
                #print(n.Value)
                if keyboard.is_pressed('o'):        # press 0 to exit the while loop
                    arr=[]
                    break
            if not arr:
                break
    else:
        pass

    if cv2.waitKey(1) & 0xFF == ord('q'):

        first_frame= False
        

#cv2.imshow("Showing contour", nf_color)
#print(arr)  
#k=cv2.waitKey(0)
cv2.destroyAllWindows()

1644
[]
[]
